In [116]:
from Calibration import *
from utils import *

In [1]:

def generate_array_from_raster(str_path_to_rasterfile, str_path_to_shapefile=None):
    if str_path_to_shapefile != None:
        with fiona.open(str_path_to_shapefile, "r") as shapefile:
            shapefile = [feature["geometry"] for feature in shapefile]

        with rasterio.open(str_path_to_rasterfile) as src:
            out_image, out_transform = rasterio.mask.mask(src, shapefile, crop=True)
#             out_image = out_image[out_image <97]
            out_image = ma.masked_values(out_image, 0)
            out_image = out_image.astype(np.float32)
            out_image = out_image.filled(np.nan)
            arr = np.expand_dims(out_image.flatten(), 0).T    
    else:
        raster = rasterio.open(str_path_to_rasterfile)
        arr = raster.read(masked=True)
        arr = np.expand_dims(arr.flatten(), 0).T    
    return arr

def get_elevations_from_raster(raster_array_flattened, nbands):
   
    n, bins, patches = plt.hist(raster_array_flattened, bins=nbands)
    plt.close()
    tot_pixels = n.sum()
#     raster_array_flattened = raster_array_flattened.filled(np.nan)
    av_elevation = round(np.nanmean(raster_array_flattened),9)
#     av_elevation = raster_array_flattened.mean()
    elevation_list = []
    for i in range(nbands):
        elevation_percentage = n[i] / tot_pixels
        elevation_list.append(round(elevation_percentage, 9))
    min_elevation = np.min(bins)
    max_elevation = np.max(bins)
    return elevation_list, bins, min_elevation, max_elevation, av_elevation

def get_landuse_from_raster(raster_array_flattened):
    n, bins, patches = plt.hist(raster_array_flattened, bins=np.arange(96))
    plt.close()
    
    tot_pixels = n.sum()
    
    bare = np.sum(n[[12, 22, 22, 24, 31]]) / tot_pixels 
    forest = np.sum(n[40:46]) / tot_pixels 
    grass = (np.sum(n[50:83])+ n[21]) /tot_pixels 
    rip = np.sum(n[[11, 90, 94]]) /tot_pixels  
    
    

    landuse_list = [round(bare,9), round(forest,9), round(grass,9), round(rip,9)] 
    return landuse_list

def generate_shapefiles_per_landuse(landuse, str_path_to_rasterfile, str_path_to_shapefile=None):  
        
    with rasterio.Env():
        if str_path_to_shapefile != None:
            with fiona.open(str_path_to_shapefile, crs='epsg:4326') as shapefile:
                shapefile = [feature["geometry"] for feature in shapefile]
            with rasterio.open(str_path_to_rasterfile) as src:
                image, out_transform = rasterio.mask.mask(src, shapefile, crop=True)
                image = ma.masked_values(image, 0)
                image = image.astype(np.float32)
                image = image.filled(np.nan)

        else:
            with rasterio.open(str_path_to_rasterfile) as src:
                image = src.read() # first band
        if landuse == 'glacier':
            image[(image == 12)] = -9999
        if landuse == 'bare':
            image[(image == 12) | (image == 22) | (image == 23)  | (image == 24) | (image == 31)] = -9999
        if landuse == 'forest':
            image[(image == 41) | (image == 42) | (image == 43)] = -9999
        if landuse == 'grass':
            image[(image == 21) | (image == 51) | (image == 52) | (image == 71) | (image == 72) | (image == 73) | (image == 74) | (image == 81) | (image == 82)] = -9999
        if landuse == 'rip':
            image[(image == 11) | (image == 90) | (image == 95)] = -9999


        if str_path_to_shapefile != None:
            results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(
                shapes(image, mask=(image ==-9999), transform= out_transform)))
        else:
            results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(
                shapes(image, mask=(image ==-9999), transform= src.transform)))            
    geoms = list(results)
    gpd_polygonized_raster  = gp.GeoDataFrame.from_features(geoms, crs='epsg:4326')
    return gpd_polygonized_raster

def clip_elevation_per_landuse(gpd_polygonized_raster, str_path_to_elevation, str_path_to_shapefile=None, export_tif=False):
    gdf = gpd_polygonized_raster.dissolve()
    coords = [json.loads(gdf.to_json())['features'][0]['geometry']] 
    if str_path_to_shapefile != None:

        with fiona.open(str_path_to_shapefile) as shapefile:
            shapefile = [feature["geometry"] for feature in shapefile]
        with rasterio.open(str_path_to_elevation) as src:
#             out_img, out_transform = rasterio.mask.mask(src, shapefile, crop=True)
            out_img, out_transform = rasterio.mask.mask(src, coords, crop=True)
            out_img = ma.masked_values(out_img, 0)
            out_img = out_img.astype(np.float32)
            out_img = out_img.filled(np.nan) 
    
    else:
        data = rasterio.open(str_path_to_elevation, shapefile=None)
        out_img, out_transform = mask(data, shapes=coords)
        out_img = np.expand_dims(out_img.flatten(), 0).T
        out_img = ma.masked_values(out_img, data.nodata)
    
    if export_tif == True:
        out_meta = data.meta.copy()
        with rasterio.open('elevationlanduse.tif', "w", **out_meta) as dest:
            dest.write(out_img)
    
    return out_img 

def generate_landuse_per_elevation(str_path_to_elevation, str_path_to_landuse, str_path_to_shapefile=None, nbands=4):
    elevation_array = generate_array_from_raster(str_path_to_elevation, str_path_to_shapefile)
    landuse_array = generate_array_from_raster(str_path_to_landuse, str_path_to_shapefile)
    
    elevation_list, bins, min_elevation, max_elevation, av_elevation = get_elevations_from_raster(elevation_array, nbands=nbands)
    landuse_catchment = get_landuse_from_raster(landuse_array)
    
    tot_elevations = [[elevation_list, landuse_catchment, (max_elevation-min_elevation)/nbands, min_elevation, max_elevation, av_elevation]]
    landuse = ['glacier', 'bare', 'forest', 'grass', 'rip']
    for x in landuse:
        gpd_polygonized_raster = generate_shapefiles_per_landuse(x, str_path_to_landuse, str_path_to_shapefile)
        if not gpd_polygonized_raster.empty:
            arr = clip_elevation_per_landuse(gpd_polygonized_raster, str_path_to_elevation, str_path_to_shapefile)
            raster_array_flattened = np.expand_dims(arr.flatten(), 0).T
            elevation_list, bins, min_elevation, max_elevation, av_elevation = get_elevations_from_raster(raster_array_flattened, nbands=nbands)
        if gpd_polygonized_raster.empty:
            elevation_list = list(np.zeros(4))
        if x == 'glacier':
            tot_elevations.append([elevation_list, list(np.around(np.array(elevation_list)*landuse_catchment[0], 9))]) 
        else:
            tot_elevations.append(elevation_list)

#         tot_elevations.append([elevation_list])
    return tot_elevations

In [2]:
el= generate_landuse_per_elevation("Data/dem_thundercreek_full.tif", "Data/NLCD_2001_Landuse.tif", "Data/Shapes/Thundercreek.shp", nbands=4)

NameError: name 'fiona' is not defined

In [252]:
gpd.read_file('Data/Shapes/Thundercreek.shp')

# stationbasins

638.0

In [114]:
directory = 'Data\Youghiogheny'
 
# iterate over files in
# that directory
dicts = {}
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    forcing = nc.Dataset(f)
    model_name = forcing.parent_source_id
    df = generate_forcing_from_NETCDF(forcing)
    dicts[model_name] = df
# generate_forcing_from_NETCDF(forcing_netcdf)

C:\Users\Vincent Hoogelander\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == '__main__':
C:\Users\Vincent Hoogelander\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == '__main__':
C:\Users\Vincent Hoogelander\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == '__main__':
C:\Users\Vincent Hoogelander\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == '__main__':
C:\Users\Vincent Hoogelander\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == '__main__':
C:\Users\Vincent Hoogelander\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning

In [115]:
dicts

{'ACCESS-ESM1-5':                 temp       prec
 2070-01-01  1.067596   0.368413
 2070-01-02  0.114716   0.195629
 2070-01-03 -0.619324   0.043563
 2070-01-04  2.906311  20.932033
 2070-01-05  8.753693   2.295993
 ...              ...        ...
 2071-12-27  6.291321   7.912514
 2071-12-28  3.610413   2.117188
 2071-12-29  0.313751   0.840211
 2071-12-30  0.093170   0.413757
 2071-12-31 -0.477722   0.166650
 
 [730 rows x 2 columns],
 'AWI-CM-1-1-MR':                  temp       prec
 2070-01-01  17.082428  11.690399
 2070-01-02  10.544434   9.728692
 2070-01-03  16.217560  24.057886
 2070-01-04   7.918518  24.469252
 2070-01-05  -3.883881   1.347589
 ...               ...        ...
 2071-12-27  -3.021545   0.792538
 2071-12-28  -4.320435   2.925353
 2071-12-29  -4.246338   0.428444
 2071-12-30  -0.802490   3.856490
 2071-12-31  -2.286255   0.675420
 
 [730 rows x 2 columns],
 'BCC-CSM2-MR':                 temp       prec
 2070-01-01  3.781616   1.134845
 2070-01-02  1.134674   0.7

In [55]:
# dicts['TaiESM1']['ACCESS-ESM1-5']

foodict = {k: v for k, v in dicts.items() if ((k == 'IPSL-CM6A-LR') | (k == 'CNRM-CM6-1-HR') | (k == 'CNRM-ESM2-1') | (k == 'CNRM-CM6-1'))}

In [69]:
x = list(dicts.values())

In [61]:
x[0].prec.mean(), x[1].prec.mean(), x[2].prec.mean(), x[3].prec.mean(), 

(3.8902120508951152,
 3.2903002645427626,
 3.6316468929823067,
 3.9612730892589862)

In [109]:

forcing2 = nc.Dataset('Data/Youghiogheny/HBVmountain_IITM-ESM_Youghiogheny_2070_2071_ssp585.nc')
forcing = nc.Dataset('Data/Youghiogheny/HBVmountain_HadGEM3-GC31-LL_Youghiogheny_2070_2071_ssp585.nc')

df = generate_forcing_from_NETCDF(forcing2)

2071-01-01 00:00:00
2071-01-01 00:00:00


C:\Users\Vincent Hoogelander\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.
C:\Users\Vincent Hoogelander\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """
C:\Users\Vincent Hoogelander\anaconda3\lib\site-pack

In [107]:
df

,temp,prec
2071-01-01,-5.545746,1.440191
2071-01-02,1.154846,0.682408
2071-01-03,0.674194,0.282793
2071-01-04,2.879883,2.260403
2071-01-05,8.560364,5.248668
...,...,...
2071-12-28,-3.092194,0.342287
2071-12-29,-0.382935,0.712998
2071-12-30,-1.006744,1.301411
2071-12-31,-2.398956,3.908589


In [108]:
def generate_forcing_from_NETCDF(forcing_netcdf):


    prec = (forcing_netcdf['pr'][:])*86400
    temp = (forcing_netcdf['tas'][:]) -273
    days = (forcing_netcdf['time'][:])

    start = date(1850,1,1)      # Netcdf file counts days from this date
  
    
    preclist = []
    tlist = []
    date_list=[]
    for i in range(len(temp)):
        preclist.append(prec[i])
        tlist.append(temp[i])
        delta = timedelta(days[i])
        date_list.append(start+delta)
    

    forcing = pd.DataFrame(index=date_list)
    forcing['temp'] = tlist
    forcing['prec'] = preclist
    forcing.index = pd.to_datetime(forcing.index)
    
    forcing.loc[forcing['prec'] > 500, 'prec'] = 0  #remove outliers 
    freq = pd.offsets.Hour(5)
    if freq.is_year_start((forcing.index[0])) == False:
        start = forcing.index[0] + pd.offsets.YearBegin()
        print(start)
        forcing = forcing.loc[start:]
    if freq.is_year_end((forcing.index[-1])) == False:
        end = forcing.index[-1] - pd.offsets.YearBegin()
        print(end)
        forcing = forcing.loc[:end][0:-1]
    forcing.index = pd.to_datetime(forcing.index).date
    return forcing